In [ ]:
import json
import glob
import string
import numpy as np
from skimage import io
from skimage.feature import match_template

In [ ]:
data_dir = 'data/'
template_dir = 'templates/'
moon_dir = data_dir+'moon/'
moon_template_dir = template_dir+'moon/'

In [ ]:
with open(moon_dir+'moon_metadata.json') as metadata:
    moon_metadata = json.load(metadata)

In [ ]:
moon_images = glob.glob(moon_dir+'*.jpg')

In [ ]:
moon_template_images = glob.glob(moon_template_dir+'[0-9]*.jpg')

In [ ]:
def match_images(filenames, templates, threshold=0.5):
    matches = {template: [] for template in templates}
    
    template_images = []
    for template in templates:
        template_images.append(io.imread(template))

    for filename in filenames:
        match_results = []
        image = io.imread(filename, as_grey=True)
        for template in template_images:
            match_results.append(match_template(image, template))
        
        # Find the template with the greatest match.
        match_results = np.array(match_results)
        max_idx = np.unravel_index(match_results.argmax(), match_results.shape)
        if match_results[max_idx] > threshold:
            template_match = templates[max_idx[0]]
            image_id = filename.translate(None, string.letters).translate(None, string.punctuation)
            matches[template_match].append(image_id)
    
    return matches

In [ ]:
moon_template_matches = match_images(moon_images, moon_template_images)